In [51]:
%load_ext autoreload
%autoreload 2

import os
import tqdm
import numpy as np
import pandas as pd
import softclustering as sc
import matplotlib.pyplot as plt
import socceraction.spadl as spadl


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Concatenate actions of all games in one DataFrame.

In [52]:
datafolder = "data"
fifa2018h5 = os.path.join(datafolder, "spadl-fifa2018.h5")
games = pd.read_hdf(fifa2018h5, key="games")
with pd.HDFStore(fifa2018h5) as store:
    actions = []  #list of DataFrames
    for game in tqdm.tqdm(games.itertuples()):
        game_action = store[f"actions/game_{game.game_id}"]
        game_action = spadl.play_left_to_right(game_action, game.home_team_id)
        game_action["is_home"] = game_action["team_id"] == game.home_team_id
        actions.append(game_action)
    actions = pd.concat(actions)
    actions.drop("original_event_id", axis=1, inplace=True)
    actions = pd.merge(actions, spadl.config.actiontypes_df(), how="left")

64it [00:00, 202.37it/s]


In [53]:
def consolidate(actions):
    #actions.fillna(0, inplace=True)

    #Consolidate corner_short and corner_crossed
    corner_idx = actions.type_name.str.contains("corner")
    actions["type_name"] = actions["type_name"].mask(corner_idx, "corner")

    #Consolidate freekick_short, freekick_crossed, and shot_freekick
    freekick_idx = actions.type_name.str.contains("freekick")
    actions["type_name"] = actions["type_name"].mask(freekick_idx, "freekick")

    #Consolidate keeper_claim, keeper_punch, keeper_save, keeper_pick_up
    keeper_idx = actions.type_name.str.contains("keeper")
    actions["type_name"] = actions["type_name"].mask(keeper_idx, "keeper_action")

    actions["start_x"] = actions["start_x"].mask(actions.type_name == "shot_penalty", 94.5)
    actions["start_y"] = actions["start_y"].mask(actions.type_name == "shot_penalty", 34)

    return actions


actions = consolidate(actions)

In [54]:
#Actions of Team France matches.
len(actions[actions["team_id"] == 771])

6829

In [55]:
actions.groupby("type_name").size()

type_name
bad_touch         1547
clearance         2074
corner             558
cross             1305
dribble          52731
foul              1876
freekick          1272
goalkick           677
interception      1681
keeper_action      584
pass             56438
shot              1556
shot_penalty        68
tackle            1830
take_on           2109
throw_in          2178
dtype: int64

As suggested in SoccerMix, add noise on the starting and ending locations, but only on those actions that we can visually note a predefined pattern.
* *Add noise in both start and end locations*:
    * Cross
    * Shot
    * Keeper_action
* *Only on start locations*:
    * Clearance
    * Goal kick
* *Only on end locations*:
    * Corner
    * Freekick
    * Shot_penalty

In [56]:
def add_noise(actions):
    # Start locations
    start_list = ["cross", "shot", "keeper_action", "clearance", "goalkick"]
    mask = actions["type_name"].isin(start_list)
    noise = np.random.normal(0, 0.5, size=actions.loc[mask, ["start_x", "start_y"]].shape)
    actions.loc[mask, ["start_x", "start_y"]] += noise

    # End locations
    end_list = ["cross", "shot", "keeper_action", "corner", "freekick", "shot_penalty"]
    mask = actions["type_name"].isin(end_list)
    noise = np.random.normal(0, 0.5, size=actions.loc[mask, ["end_x", "end_y"]].shape)
    actions.loc[mask, ["end_x", "end_y"]] += noise

    return actions


actions = add_noise(actions)

In [57]:
# # display event locations with noise
# corrected_actions = ["cross", "shot", "keeper_action", "clearance", "goalkick","corner", "freekick", "shot_penalty"]
# for actiontype in corrected_actions:
#     actions[actions.type_name == actiontype].plot.scatter(
#         x="start_x",
#         y="start_y",
#         title = f"Start Location: {actiontype}",
#         figsize = (6,4)
#     )
#     plt.show()
#     actions[actions.type_name == actiontype].plot.scatter(
#         x="end_x",
#         y="end_y",
#         title = f"End Location: {actiontype}",
#         figsize = (6,4)
#     )
#     plt.show()

Compute the angle of the direction of the action with respect with the x-axis (pitch's length) a
$$\tan \theta = \frac{y_{end} - y_{start}}{x_{end} - x_{start}}$$

In [58]:
actions["angle"] = np.arctan2(actions.end_y - actions.start_y, actions.end_x - actions.start_x)
actions["cos_angle"] = np.cos(actions["angle"])
actions["sin_angle"] = np.sin(actions["angle"])
actions[["angle", "cos_angle", "sin_angle"]].describe()

,angle,cos_angle,sin_angle
count,128484.000000,128484.000000,128484.000000
mean,0.062299,0.313633,-0.005792
std,1.464611,0.678503,0.664260
min,-3.139783,-1.000000,-1.000000
25%,-0.969342,-0.183971,-0.647648
50%,0.000000,0.525493,0.000000
75%,1.076271,0.954549,0.624695
max,3.141593,1.000000,1.000000


In [59]:
mask = (actions["type_name"]=="throw_in") & (actions["team_id"]==771)
data_loc = actions[mask][["start_x", "start_y"]]
data_loc.describe()

,start_x,start_y
count,116.000000,116.000000
mean,60.918103,38.030172
std,26.507723,33.145853
min,9.187500,0.425000
25%,38.937500,0.425000
50%,64.750000,66.725000
75%,85.531250,67.575000
max,101.937500,67.575000


In [60]:
K_gauss = 6
gauss_clusters = [sc.MultivariateGaussian() for j in range(K_gauss)]
gaussian_model = sc.MixtureModel(gauss_clusters)
_ = gaussian_model.fit_classical_EM(data_loc, verbose=True)

Data log-likelihood at iter 0: -661.97
Data log-likelihood at iter 1: -660.48
Data log-likelihood at iter 2: -660.00
Data log-likelihood at iter 3: -659.66
Data log-likelihood at iter 4: -659.36
Data log-likelihood at iter 5: -659.10
Data log-likelihood at iter 6: -658.87
Data log-likelihood at iter 7: -658.68
Data log-likelihood at iter 8: -658.50
Data log-likelihood at iter 9: -658.33
Data log-likelihood at iter 10: -658.15
Data log-likelihood at iter 11: -657.96
Data log-likelihood at iter 12: -657.75
Data log-likelihood at iter 13: -657.52
Data log-likelihood at iter 14: -657.27
Data log-likelihood at iter 15: -657.01
Data log-likelihood at iter 16: -656.71
Data log-likelihood at iter 17: -656.38
Data log-likelihood at iter 18: -655.96
Data log-likelihood at iter 19: -655.33
Data log-likelihood at iter 20: -653.98
Data log-likelihood at iter 21: -649.49
Data log-likelihood at iter 22: -621.18
Data log-likelihood at iter 23: -512.85
Data log-likelihood at iter 24: -448.19
Data log-l

In [61]:
for cluster in gaussian_model.components:
    print(cluster)

MultivariateGaussian(d=2, mean=[20.79851758  1.275     ], cov=[[3.22870946e+01 0.00000000e+00]
 [0.00000000e+00 1.00000000e-09]])
MultivariateGaussian(d=2, mean=[56.52770319 67.05891077], cov=[[60.89060947 -2.50906618]
 [-2.50906618  0.17232775]])
MultivariateGaussian(d=2, mean=[64.98333333  0.425     ], cov=[[ 6.78174462e+02 -3.69778549e-32]
 [-6.76503222e-32  1.00000000e-09]])
MultivariateGaussian(d=2, mean=[83.35255752 67.439553  ], cov=[[94.31317171 -0.65660458]
 [-0.65660458  0.09678406]])
MultivariateGaussian(d=2, mean=[25.77454388 67.10292548], cov=[[106.56115203  -1.34938545]
 [ -1.34938545   0.17840899]])
MultivariateGaussian(d=2, mean=[64.65820656  1.275     ], cov=[[3.52130206e+02 1.03537994e-30]
 [9.38905311e-31 1.00000000e-09]])


In [62]:
data_dir = actions[mask][["cos_angle","sin_angle"]]
K_vm = 4
vm_clusters = [sc.VonMises() for j in range(K_vm)]
vonmises_model = sc.MixtureModel(vm_clusters)
_ = vonmises_model.fit_classical_EM(data_dir, verbose=True)

Data log-likelihood at iter 0: -183.78
Data log-likelihood at iter 1: -183.20
Data log-likelihood at iter 2: -183.08
Data log-likelihood at iter 3: -183.04
Data log-likelihood at iter 4: -183.01
Data log-likelihood at iter 5: -182.99
Data log-likelihood at iter 6: -182.98
Data log-likelihood at iter 7: -182.97
Data log-likelihood at iter 8: -182.96
Data log-likelihood at iter 9: -182.96
Data log-likelihood at iter 10: -182.95
Data log-likelihood at iter 11: -182.94
Data log-likelihood at iter 12: -182.94
Data log-likelihood at iter 13: -182.93
Data log-likelihood at iter 14: -182.93
Data log-likelihood at iter 15: -182.92
Data log-likelihood at iter 16: -182.92
Data log-likelihood at iter 17: -182.92
Data log-likelihood at iter 18: -182.91
Data log-likelihood at iter 19: -182.91
Data log-likelihood at iter 20: -182.91
Data log-likelihood at iter 21: -182.90
Data log-likelihood at iter 22: -182.90
Data log-likelihood at iter 23: -182.90
Data log-likelihood at iter 24: -182.90
Data log-l

In [63]:
for cluster in vonmises_model.components:
    print(cluster)

VonMises(loc=-5.7º, kappa=12.692)
VonMises(loc=173.4º, kappa=12.948)
VonMises(loc=-103.3º, kappa=4.191)
VonMises(loc=93.3º, kappa=2.296)
